In [1]:
pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Import all required packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from sklearn.model_selection import train_test_split
import os

# Import TensorFlow packages 

import tensorflow as tf   
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Conv2D, MaxPooling2D





In [3]:
#to use functions from scripts in the image processing folder
import sys
from pathlib import Path
sys.path.insert(0, str(Path(sys.path[0]).parent.absolute()))
from EDA.EDA_functions import *
from preprocessing.image_processing import load_MRI_gifs, extract_box, stacked_boxes

In [4]:
#Load the images 
df=pd.read_csv('/Users/tamarapallien/neuefische/capstone /ds-capstone-alzheimers-/Data/Tables/oasis_cross-sectional.csv')
df.head(2)

,ID,M/F,Hand,Age,Educ,SES,MMSE,CDR,eTIV,nWBV,ASF,Delay
0,OAS1_0001_MR1,F,R,74,2.0,3.0,29.0,0.0,1344,0.743,1.306,NaN
1,OAS1_0002_MR1,F,R,55,4.0,1.0,29.0,0.0,1147,0.810,1.531,NaN


In [5]:
#Replace NaN in CDR column with 0 
df['CDR'].fillna(0, inplace=True)

In [6]:
## image loading
def load_MRI_gifs(IDs, plane="all"):
    '''Given a set of IDs ("OAS1_0xxx_MR1") returns the atlas corrected traverse, sagittal and cortical images as numpy arrays'''
    tra = []
    sag = []
    cor = []
    for path in IDs:
        path1 = '../Data/Images/' + path + '/PROCESSED/MPRAGE/T88_111/'
        for img in os.listdir(path1):
            if img.endswith('t88_gfc_tra_90.gif'):
                tra.append(plt.imread(path1+img))
            elif img.endswith('t88_gfc_sag_95.gif'):
                sag.append(plt.imread(path1+img))
            elif img.endswith('t88_gfc_cor_110.gif'):
                cor.append(plt.imread(path1+img))
    if plane == "traverse":
        return np.array(tra)
    elif plane == "sagittal":
        return np.array(sag)
    elif plane == "cortical":
        return np.array(cor)
    elif plane == "all":
        return [np.array(tra), np.array(sag), np.array(cor)]
    else:
        print("Inncorect selection of planes")

In [7]:
#Load the single slice images 
tra, sag, cor= load_MRI_gifs(df["ID"], "all")

In [8]:
#create all labels
labels=df['CDR']!=0
labels=labels.astype(int) #change the bools to integer

In [9]:
#Train, test split 

X_train, X_test, y_train, y_test=train_test_split(tra,labels, random_state=42, stratify=labels)

In [10]:
image_shape=tra.shape
image_shape

(436, 208, 176)

In [11]:
# Built a model 
model=Sequential()
model.add(InputLayer(input_shape=[208,176]))
model.add(Flatten())
model.add(Dense(units=20, activation='relu'))
model.add(Dense(units=1, activation='softmax'))


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-08-19 14:52:08.861584: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-19 14:52:08.862238: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
#Compile model 
import tensorflow_addons as tfa

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy', keras.metrics.Recall(), tfa.metrics.MatthewsCorrelationCoefficient(num_classes=2)])

In [13]:
#Train the model 
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=True, validation_split=0.2)

Epoch 1/10


2022-08-19 14:52:09.180797: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-19 14:52:09.789704: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - ETA: 0s - loss: 86.5943 - accuracy: 0.2299 - recall: 1.0000 - MatthewsCorrelationCoefficient: 0.0000e+00

2022-08-19 14:52:10.826013: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 2s 80ms/step - loss: 86.5943 - accuracy: 0.2299 - recall: 1.0000 - MatthewsCorrelationCoefficient: 0.0000e+00 - val_loss: 0.6926 - val_accuracy: 0.2273 - val_recall: 1.0000 - val_MatthewsCorrelationCoefficient: 0.0000e+00
Epoch 2/10
9/9 [==============================] - 0s 32ms/step - loss: 0.6923 - accuracy: 0.2299 - recall: 1.0000 - MatthewsCorrelationCoefficient: 0.0000e+00 - val_loss: 0.6910 - val_accuracy: 0.2273 - val_recall: 1.0000 - val_MatthewsCorrelationCoefficient: 0.0000e+00
Epoch 3/10
9/9 [==============================] - 0s 40ms/step - loss: 0.6905 - accuracy: 0.2299 - recall: 1.0000 - MatthewsCorrelationCoefficient: 0.0000e+00 - val_loss: 0.6891 - val_accuracy: 0.2273 - val_recall: 1.0000 - val_MatthewsCorrelationCoefficient: 0.0000e+00
Epoch 4/10
9/9 [==============================] - 0s 40ms/step - loss: 0.6886 - accuracy: 0.2299 - recall: 1.0000 - MatthewsCorrelationCoefficient: 0.0000e+00 - val_loss: 0.6870 - val_accuracy: 0.2

In [15]:
#Evaluate the model 
(y_pred>0.5).astype(int).max()

1

In [28]:
model2=Sequential()
model2.add(Conv2D(2,3,activation='relu', padding='same', input_shape=(208, 176,1)))
model2.add(Conv2D(2,3, activation='relu', padding='same'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Conv2D(2,3, activation='relu', padding='same'))
model2.add(Conv2D(2,3, activation='relu', padding='same'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Flatten())
model2.add(Dropout(0.15))
model2.add(Dense(100, activation='relu'))
model2.add(Dropout(0.15))
model2.add(Dense(50, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

In [29]:
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [30]:
model2.fit(X_train, y_train, batch_size=32, epochs=50, verbose=True, validation_split=0.2)

Epoch 1/50


2022-08-19 14:59:14.348496: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 1s 91ms/step - loss: 0.0000e+00 - accuracy: 0.7663 - val_loss: 0.0000e+00 - val_accuracy: 0.7727
Epoch 2/50
1/9 [==>...........................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.8125

2022-08-19 14:59:15.216854: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 0s 35ms/step - loss: 0.0000e+00 - accuracy: 0.7701 - val_loss: 0.0000e+00 - val_accuracy: 0.7727
Epoch 3/50
9/9 [==============================] - 0s 30ms/step - loss: 0.0000e+00 - accuracy: 0.7701 - val_loss: 0.0000e+00 - val_accuracy: 0.7727
Epoch 4/50
9/9 [==============================] - 0s 30ms/step - loss: 0.0000e+00 - accuracy: 0.7701 - val_loss: 0.0000e+00 - val_accuracy: 0.7727
Epoch 5/50
9/9 [==============================] - 0s 31ms/step - loss: 0.0000e+00 - accuracy: 0.7701 - val_loss: 0.0000e+00 - val_accuracy: 0.7727
Epoch 6/50
9/9 [==============================] - 0s 30ms/step - loss: 0.0000e+00 - accuracy: 0.7701 - val_loss: 0.0000e+00 - val_accuracy: 0.7727
Epoch 7/50
9/9 [==============================] - 0s 31ms/step - loss: 0.0000e+00 - accuracy: 0.7701 - val_loss: 0.0000e+00 - val_accuracy: 0.7727
Epoch 8/50
9/9 [==============================] - 0s 29ms/step - loss: 0.0000e+00 - accuracy: 0.7701 - val_loss: 0.0000e+00 - val

In [31]:
y_pred=model2.predict(X_test)

2022-08-19 15:23:56.376751: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
